In [1]:
# Get the dataset from Github
!wget --no-check-certificate \
    "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt" \
    -O "/tmp/words.txt"

--2023-03-26 04:47:07--  https://raw.githubusercontent.com/dwyl/english-words/master/words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4862992 (4.6M) [text/plain]
Saving to: ‘/tmp/words.txt’

/tmp/words.txt      100%[===================>]   4.64M  --.-KB/s    in 0.09s   

2023-03-26 04:47:07 (51.6 MB/s) - ‘/tmp/words.txt’ saved [4862992/4862992]

